In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field

In [21]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [22]:
embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="bge-m3:latest")
vectorstore = Chroma(persist_directory="./db/chroma_index", embedding_function=embed_model)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000029882899360>, search_kwargs={'k': 3})

In [51]:
question = " generator?"
docs = retriever.invoke(question)
docs

[Document(metadata={'File Name': 'FWG', 'File Path': '.\\documents\\pos\\FWG.pdf', 'First Division': 'documents', 'Page': 0, 'Second Division': 'pos'}, page_content='|None||None|None|None|F.W. GENERATOR|None|None|None|None|\n|TEL. NO.|SHIP NO.|None|None|None|None|None|None|None|None|\n|*1-6325|8250/8251|None|None|None|None|None|None|None|None|\n||APPROVED|None|Hyunseung Kim||None|None|None|None|None|\n|None||None||||None|None|None|None|\n|None|CHECKED|None|Seungho Han|||None|None|None|None|\n|None||None||||None|None|None|None|\n|None|DRAWN|None|Junsuk Yoon||REFERENCE DRAWING NO. 8250536-0|None|None|DRAWING SCALE|None|\n|HYUNDAI HEAVY INDUSTRIES CO., LTD. SAMHO SHIPYARD. KOREA|None|None|None|None|None|None|None|None|None|\n|None|None|None|None|None|SEQUENCE NO.|None||None|None|\n|None|None|None|None|None||None|None|None|None|'),
 Document(metadata={'File Name': 'FW Generator', 'File Path': '.\\documents\\pos\\FW Generator.pdf', 'First Division': 'documents', 'Page': 0, 'Second Division'

In [52]:
# Prompt   (rlm/rag-prompt)
prompt = ChatPromptTemplate.from_messages([
    ("human", 
    """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Question: {question} 
    Context: {context} 
    Answer:"""),
    ])
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \n    Use the following pieces of retrieved context to answer the question. \n    If you don't know the answer, just say that you don't know. \n    Use three sentences maximum and keep the answer concise.\n    Question: {question} \n    Context: {context} \n    Answer:"), additional_kwargs={})])

In [53]:
# llm = ChatGroq(temperature=0, model_name= "llama-3.2-90b-text-preview")
llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:latest")

In [54]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [55]:
total_docs = format_docs(docs)
total_docs

"|None||None|None|None|F.W. GENERATOR|None|None|None|None|\n|TEL. NO.|SHIP NO.|None|None|None|None|None|None|None|None|\n|*1-6325|8250/8251|None|None|None|None|None|None|None|None|\n||APPROVED|None|Hyunseung Kim||None|None|None|None|None|\n|None||None||||None|None|None|None|\n|None|CHECKED|None|Seungho Han|||None|None|None|None|\n|None||None||||None|None|None|None|\n|None|DRAWN|None|Junsuk Yoon||REFERENCE DRAWING NO. 8250536-0|None|None|DRAWING SCALE|None|\n|HYUNDAI HEAVY INDUSTRIES CO., LTD. SAMHO SHIPYARD. KOREA|None|None|None|None|None|None|None|None|None|\n|None|None|None|None|None|SEQUENCE NO.|None||None|None|\n|None|None|None|None|None||None|None|None|None|\n\n|None||None|None|None|F.W. GENERATOR|None|None|None|None|\n|TEL. NO.|SHIP NO.|None|None|None|None|None|None|None|None|\n|*1-6325|8250/8251|None|None|None|None|None|None|None|None|\n||APPROVED|None|Hyunseung Kim||None|None|None|None|None|\n|None||None||||None|None|None|None|\n|None|CHECKED|None|Seungho Han|||None|None|None|N

In [56]:
# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": total_docs, "question": question})
print(generation)

The context doesn't provide a clear indication of what "generator" refers to in this case. I don't know the answer to your question about "generator".
